In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from pathlib import Path

from src import preprocessamento, deteccao_pontos, reconstrucao_sfm

# Parâmetros da câmera (SIMPLE_RADIAL)
K = np.array([
    [2559.68, 0, 1536],
    [0, 2559.68, 1152],
    [0, 0, 1]
], dtype=np.float32)
dist = np.array([-0.0204997, 0, 0, 0, 0], dtype=np.float32)

img1_path = "/home/kaiooliveira/Documentos/kaio/dev/Fotogrametria/fotogrametria_aplicada/data/bruto/south-building/images/P1180142.JPG" 
img2_path = "/home/kaiooliveira/Documentos/kaio/dev/Fotogrametria/fotogrametria_aplicada/data/bruto/south-building/images/P1180143.JPG" 

img1 = preprocessamento.processar_imagem(str(img1_path), K, dist)
img2 = preprocessamento.processar_imagem(str(img2_path), K, dist)

kp1, desc1 = deteccao_pontos.detectar_pontos(img1, metodo="SIFT")
kp2, desc2 = deteccao_pontos.detectar_pontos(img2, metodo="SIFT")
matches = deteccao_pontos.corresponder_descritores(desc1, desc2, metodo="BF")

pontos_3d, R, t, _ = reconstrucao_sfm.estimar_poses_e_triangulacao(kp1, kp2, matches, K)

print(f"[INFO] Pontos triangulados: {pontos_3d.shape[0]}")
print(f"[INFO] Matriz de rotação:\n{R}")
print(f"[INFO] Vetor de translação:\n{t.ravel()}")


def mostrar_nuvem_pontos(pontos):
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pontos)
    pc.paint_uniform_color([0.1, 0.6, 0.1])
    o3d.visualization.draw_geometries([pc])

mostrar_nuvem_pontos(pontos_3d)

Total de matches: 1794
Válidos: 457470
Shape proj1: (3, 4)
Shape proj2: (3, 4)
Shape pts1_valid: (2, 1794)
Shape pts2_valid: (2, 1794)
[INFO] Pontos triangulados: 1794
[INFO] Matriz de rotação:
[[ 0.98283907  0.18188483  0.03074519]
 [-0.17938498  0.90355727  0.38910833]
 [ 0.04299286 -0.3879461   0.92067879]]
[INFO] Vetor de translação:
[-0.11161873 -0.96632637  0.23185038]
